# Penmanshiel wind farm data exploration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import folium
import os
from elucidata.resources.pipeline import DataFrameDownload

In [ ]:
data_path = os.path.join('..', 'data', 'Penmanshiel')

## Fleet information

In [ ]:
local_file = os.path.join(data_path, 'Penmanshiel_WT_static.csv')
df_info = DataFrameDownload(local_file,
                            'outputs/FAIR2/Penmanshiel/Penmanshiel_WT_static.csv', 
                            skiprows=range(15, 100)).make().dropna(axis=1)

In [ ]:
df_info

The dataset consists of 14 turbines (T03 is missing), they all have the same specifications and started operations. From the coordinates and the fact that they are referred to as a "farm", we assume they are co-located. We confirm this by plotting them on map:

In [ ]:
center_loc = df_info[['Latitude', 'Longitude']].mean().values

m = folium.Map(location=center_loc, zoom_start=13)

for _j, row in df_info.iterrows():
    folium.Marker(
    location=row[['Latitude', 'Longitude']].values, # coordinates for the marker (Earth Lab at CU Boulder)
    popup=f"{row['Alternative Title']} ({row['Elevation (m)']}m)", # pop-up label for the marker
    icon=folium.Icon()
).add_to(m)

m

They are indeed co-located in a hilly area in the south of Scotland. As can be seen in the elevation values for each turbine, these can differ substantially, with a maximum difference of 48 meters. This is an important factor to keep in mind.

## Scada data

In [ ]:
def read_data_penmanshiel(turbine_number):
    
    local_file_scada = os.path.join(data_path, f'scada_T{turbine_number:02d}.csv')
    df_scada = DataFrameDownload(local_file_scada,
                                 f'outputs/FAIR2/Penmanshiel/scada_T{turbine_number:02d}.csv').make()
    df_scada = df_scada.set_index('Datetime')
    
    local_file_logs = os.path.join(data_path, f'logs_T{turbine_number:02d}.csv')
    df_logs = DataFrameDownload(local_file_logs,
                                 f'outputs/FAIR2/Penmanshiel/logs_T{turbine_number:02d}.csv').make()

    return df_scada, df_logs

### Turbine 1
As an example, we'll look into Turbine 1.

In [ ]:
df_t1, df_logs_t1 = read_data_penmanshiel(1)

In [ ]:
df_t1

In [ ]:
# Plot the power curve
sns.scatterplot(data=df_t1, x='Wind speed (m/s)', y='Power (kW)')

We see a typical power curve, with some clear curtailment modes as well.

Now let's have a look at the sensors in the dataset. The dataframe has 363 columns. However, most of these are related to the same quantity measured by the SCADA system, reporting the average, standard deviation, minimum, and maximum. We filter out all the columns corresponding to the STD, min, and max (which always have a ',' in their column name). 

In [ ]:
sensors = [col for col in df_t1.columns if ',' not in col]
print(len(sensors))
sensors

We find a long list of sensors (147), not all of which are clear what they mean, some probably not very useful, others highly correlated to others.

Let's inspect the theoretical energy and compare it to the export energy:

In [ ]:
sns.scatterplot(data=df_t1, x='Wind speed (m/s)', y='Energy Export (kWh)')
sns.scatterplot(data=df_t1, x='Wind speed (m/s)', y='Energy Theoretical (kWh)', color='g')

The theoretical energy seems less spread out, but also not a clear 1-1 with wind speed. It remains unclear what this actually quantifies.

Now, let's have a look at the time evolution of the power output. We first resample the data to 1 day.

In [ ]:
df_t1.index = pd.to_datetime(df_t1.index)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
df_sub = df_t1.resample('1D').median()
sns.lineplot(data=df_sub, x=df_sub.index, y='Power (kW)', ax=ax)

Below we show a heatmap of the missing data.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(df_t1[sensors].isna(), cmap='Grays', cbar=False, ax=ax)

## Logs

Now we look at the log files, again for turbine 1.

In [ ]:
df_logs_t1

In [ ]:
df_logs_t1['Status'].value_counts()

We see that there are many logs reported, but most of them seem inconsequential. We zoom in on the ones that caused a "Stop".

In [ ]:
df_logs_t1[df_logs_t1['Status'] == 'Stop']

1227 instances where the turbine had to be shut down. Let's see what their message was and how long they were.

In [ ]:
df_logs_t1[df_logs_t1['Status'] == 'Stop']['Message'].value_counts()

In [ ]:
df_logs_t1[df_logs_t1['Status'] == 'Stop']['Duration']

In [ ]:
def duration_to_hours(x):
    x_split = x.split(':')
    return int(x_split[0]) + int(x_split[1]) / 60  + int(x_split[2]) / 3600

df_stop = df_logs_t1[df_logs_t1['Status'] == 'Stop']
df_stop['Duration in hours'] = df_stop['Duration'].apply(lambda x: duration_to_hours(x))

fix, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
sns.boxplot(df_stop['Duration in hours'], ax=ax1)
sns.boxplot(df_stop[df_stop['Duration in hours'] < 2]['Duration in hours'], ax=ax2)


We see that most stops were less than an hour, but with a few strong outliers. We look into these.

In [ ]:
df_stop[df_stop['Duration in hours'] > 50]

Some of these correspond to maintenance. Let's look at all logs were maintenance was reported and how long they took.

In [ ]:
df_logs_maintenance = df_logs_t1[df_logs_t1['IEC category'].apply(lambda x: 'Maintenance' in str(x))]
df_logs_maintenance

In [ ]:
df_logs_maintenance['Duration in hours'] = df_logs_maintenance['Duration'].apply(lambda x: duration_to_hours(x))

In [ ]:
fix, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
sns.boxplot(df_logs_maintenance['Duration in hours'], ax=ax1)
sns.boxplot(df_logs_maintenance['Duration in hours'], ax=ax2)
ax2.set_ylim(-1, 5)

About half of the maintenance events are less than 1 hour, however some lasted several hours, upto ~5 days (120 hours). If we select the ones lasting more than 5 hours, we have 10 events left over for turbine 1.

In [ ]:
df_logs_maintenance[df_logs_maintenance['Duration in hours'] > 5]